In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
%matplotlib inline

Notebook of things to do for a SummerSim Paper on common random values

In [2]:
n_draws = 1000
n_simulants = 1000
n_days = 365
t_timestep = 1 # days
t_start = pd.Timestamp('2010-01-01')

Simple model (pencil-and-paper calculable):

In [3]:
incidence = 1.72  # per person-year
special_frac = .056

In [4]:
all_cases = incidence * (n_simulants * n_days / 365)
special_cases = all_cases * special_frac

cases_averted = special_cases
print('cases_averted:', cases_averted)

cases_averted: 96.32000000000001


Repeat to include distribution:

In [5]:
np.random.seed(12345) # set seed for reproducibility

In [6]:
mu_incidence = incidence
sigma_incidence = .12

mu_special_frac = special_frac
sigma_special_frac = .018

In [7]:
def my_truncated_normal(mu, sigma, lower=0, upper=np.inf):
    vals = np.random.normal(mu, sigma, size=n_draws)
    vals = np.clip(vals, lower, upper)
    return vals    

In [8]:
df = pd.DataFrame(index=range(n_draws))
df['incidence'] = my_truncated_normal(mu_incidence, sigma_incidence)
df['special_frac'] = my_truncated_normal(mu_special_frac, sigma_special_frac)

In [9]:
import pymc as pm

In [10]:
df['all_cases'] = df.incidence * (n_simulants * n_days / 365)
df['special_cases'] = df.all_cases * df.special_frac

df['cases_averted'] = df['special_cases']
print('cases_averted:', df.cases_averted.mean(), pm.utils.hpd(df.cases_averted, .05))

cases_averted: 96.9280773547 [  34.53660641  161.30704137]


In [11]:
lb, ub = pm.utils.hpd(df.cases_averted, .05)
print('UI width')
ub - lb

UI width


126.77043496029238

Now do a simulation version, using CEAM just because:

In [12]:
import ceam_public_health.components as cphc,\
    ceam_public_health.components.base_population
import ceam_tests.util as ctu

from ceam.framework.event import listens_for
from ceam.framework.population import uses_columns

@listens_for('initialize_simulants')
@uses_columns(['age', 'sex', 'cases'])
def my_generate_base_population(event):
    population = pd.DataFrame(index=event.index)
    population['age'] = 0
    population['sex'] = '-'
    population['cases'] = 0.

    event.population_view.update(population)

components = [my_generate_base_population]
simulation = ctu.setup_simulation(components, population_size=n_simulants, start=t_start)
ctu.pump_simulation(simulation, duration=pd.Timedelta(n_days))

simulation.population.population.cases.sum()

0.0

In [13]:
class MyDiarrhea:
    def __init__(self, incidence, special_frac, eliminated, crn):
        self.incidence = incidence
        self.special_frac = special_frac
        self.eliminated = eliminated
        self.crn = crn
    
    def setup(self, builder):
        if self.crn:
            self.rand1 = builder.randomness('any_diarrhea')
            self.rand2 = builder.randomness('special')

    @listens_for('time_step')
    @uses_columns(['cases'])
    def experience_disease(self, event):
        n = len(event.index)
        
        # FIXME: get timestep from sim, don't hardcode
        case_pr = 1. - np.exp(-self.incidence/365.)
        
        if not self.crn:
            case_indicator = np.random.uniform(size=n) < case_pr
            case_index = event.index[case_indicator]
        else:
            case_index = self.rand1.filter_for_probability(event.index, case_pr)
        
        if self.eliminated:
            not_special_pr = 1-self.special_frac
            if not self.crn:
                not_special_indicator = np.random.uniform(size=case_index.shape) < not_special_pr
                case_index = case_index[not_special_indicator]
            else:
                case_index = self.rand2.filter_for_probability(case_index, not_special_pr)
        
        
        event.population.cases[case_index] += 1

        event.population_view.update(event.population)

cases = {}
for eliminated in [False, True]:
    for crn in [True, False]:
        components = [my_generate_base_population, 
                      MyDiarrhea(incidence, special_frac, eliminated, crn)]
        simulation = ctu.setup_simulation(components, population_size=n_simulants, start=t_start)
        ctu.pump_simulation(simulation, iterations=n_days, )

        cases[eliminated] = simulation.population.population.cases.sum()
print('cases_averted:', cases[False] - cases[True])

cases_averted: 158.0


In [14]:
%%time

def replicate_sims(crn):
    cases = {True: [], False: []}
    for eliminated in [False, True]:
        for i in range(n_draws):
            incidence, special_frac = df.incidence[i], df.special_frac[i]
            components = [my_generate_base_population,
                          MyDiarrhea(incidence, special_frac, eliminated, crn)]
            simulation = ctu.setup_simulation(components, population_size=n_simulants, start=t_start)
            ctu.pump_simulation(simulation, iterations=n_days, )

            cases[eliminated].append(simulation.population.population.cases.sum())
    t = pd.DataFrame(cases)
    t['cases_averted'] = t[False] - t[True]
    return t

t = replicate_sims(crn=False)

CPU times: user 1h 9min 2s, sys: 1.12 s, total: 1h 9min 3s
Wall time: 1h 9min 5s


In [15]:
print('Without CRN')
print('cases_averted:', t.cases_averted.mean(), pm.utils.hpd(t.cases_averted, .05))

Without CRN
cases_averted: 98.859 [ -23.  234.]


In [16]:
lb, ub = pm.utils.hpd(t.cases_averted, .05)
print('UI width')
ub - lb

UI width


257.0

In [17]:
%%time

# now again, with common random values
t = replicate_sims(crn=True)
print('With CRN')
print('cases_averted:', t.cases_averted.mean(), pm.utils.hpd(t.cases_averted, .05))
lb, ub = pm.utils.hpd(t.cases_averted, .05)
print('UI width', ub - lb)

With CRN
cases_averted: 98.918 [  28.  161.]
UI width 133.0
CPU times: user 1h 15min 2s, sys: 1.14 s, total: 1h 15min 3s
Wall time: 1h 15min 5s
